<a href="https://colab.research.google.com/github/palakagl/NLP/blob/main/Wysdom_PersonalAssistant_Hybrid_XGBoost_NB_RF_CatBoost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install sentence-transformers

     |████████████████████████████████| 79 kB 7.5 MB/s 
     |████████████████████████████████| 4.0 MB 34.0 MB/s 
     |████████████████████████████████| 1.2 MB 41.5 MB/s 
     |████████████████████████████████| 77 kB 7.9 MB/s 
     |████████████████████████████████| 596 kB 71.4 MB/s 
     |████████████████████████████████| 6.6 MB 59.5 MB/s 
     |████████████████████████████████| 895 kB 62.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=fa7496df335f97d58619f340e70d1c9185b7fe04840a504db47687ef4515f954
  Stored in directory: /root/.cache/pip/wheels/83/c0/df/b6873ab7aac3f2465aa9144b6b4c41c4391cfecc027c8b07e7
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
!pip install catboost

     |████████████████████████████████| 76.6 MB 60 kB/s 


In [4]:
import datetime
import pandas as pd
import numpy as np
import os
import re

import nltk
import io
from nltk import word_tokenize
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams

from sentence_transformers import SentenceTransformer, LoggingHandler

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import catboost
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics.cluster import adjusted_rand_score, adjusted_mutual_info_score

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Dataset

In [5]:
from google.colab import files
uploaded = files.upload()

Saving personal_assistant_train.csv to personal_assistant_train.csv


In [6]:
df = pd.read_csv(io.BytesIO(uploaded['personal_assistant_train.csv']))
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8884 entries, 0 to 8883
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       8884 non-null   int64 
 1   message  8884 non-null   object
 2   label    8884 non-null   object
dtypes: int64(1), object(2)
memory usage: 208.3+ KB
None
      id                                            message             label
0  10480                           start the robotic vacuum      iot_cleaning
1   4210          i am sorry but i think you are not right.    general_negate
2   2443  what is the time difference between california...  datetime_convert
3   5869                              tell me my list names       lists_query
4   2801                 beep when i get an email from john       email_query


In [7]:
df.head()

,id,message,label
0,10480,start the robotic vacuum,iot_cleaning
1,4210,i am sorry but i think you are not right.,general_negate
2,2443,what is the time difference between california...,datetime_convert
3,5869,tell me my list names,lists_query
4,2801,beep when i get an email from john,email_query


encoding labels

In [8]:
# Encode the labels
df['encoded_label'] = df['label'].astype('category').cat.codes
b = df['label'].astype('category')

#Create the category mapping to map it back
cat_map = dict(enumerate(b.cat.categories))
df.head()

,id,message,label,encoded_label
0,10480,start the robotic vacuum,iot_cleaning,26
1,4210,i am sorry but i think you are not right.,general_negate,22
2,2443,what is the time difference between california...,datetime_convert,10
3,5869,tell me my list names,lists_query,36
4,2801,beep when i get an email from john,email_query,13


In [9]:
cat_map

{0: 'alarm_query',
 1: 'alarm_remove',
 2: 'alarm_set',
 3: 'audio_volume_down',
 4: 'audio_volume_mute',
 5: 'audio_volume_up',
 6: 'calendar_query',
 7: 'calendar_remove',
 8: 'calendar_set',
 9: 'cooking_recipe',
 10: 'datetime_convert',
 11: 'datetime_query',
 12: 'email_addcontact',
 13: 'email_query',
 14: 'email_querycontact',
 15: 'email_sendemail',
 16: 'general_affirm',
 17: 'general_commandstop',
 18: 'general_confirm',
 19: 'general_dontcare',
 20: 'general_explain',
 21: 'general_joke',
 22: 'general_negate',
 23: 'general_praise',
 24: 'general_quirky',
 25: 'general_repeat',
 26: 'iot_cleaning',
 27: 'iot_coffee',
 28: 'iot_hue_lightchange',
 29: 'iot_hue_lightdim',
 30: 'iot_hue_lightoff',
 31: 'iot_hue_lighton',
 32: 'iot_hue_lightup',
 33: 'iot_wemo_off',
 34: 'iot_wemo_on',
 35: 'lists_createoradd',
 36: 'lists_query',
 37: 'lists_remove',
 38: 'music_likeness',
 39: 'music_query',
 40: 'music_settings',
 41: 'news_query',
 42: 'play_audiobook',
 43: 'play_game',
 44

In [10]:
x=df['message']
y=df['encoded_label']

In [11]:
x.head()

0                             start the robotic vacuum
1            i am sorry but i think you are not right.
2    what is the time difference between california...
3                                tell me my list names
4                   beep when i get an email from john
Name: message, dtype: object

In [12]:
y.head()

0    26
1    22
2    10
3    36
4    13
Name: encoded_label, dtype: int8

In [13]:
from google.colab import files
uploaded = files.upload()

Saving personal_assistant_test.csv to personal_assistant_test.csv


In [14]:
df_test = pd.read_csv(io.BytesIO(uploaded['personal_assistant_test.csv']))
print(df_test.info())
print(df_test.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2221 entries, 0 to 2220
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       2221 non-null   int64 
 1   message  2221 non-null   object
dtypes: int64(1), object(1)
memory usage: 34.8+ KB
None
      id                                     message
0  10873  list the current price of microsoft shares
1   8491             where is the nearest smoke shop
2   4549                          please start over.
3   5948                         read my lists to me
4   1663                             mute volume now


In [15]:
X_test=df_test['message']

# Embeddings 

In [16]:
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2')
#embedder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

x_embeddings = embedder.encode(x, convert_to_numpy=True)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/314 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [17]:
x_embeddings

array([[-0.5887816 , -0.1500709 , -0.24277788, ...,  0.32644942,
        -0.4613069 , -0.31849873],
       [ 0.21024746, -0.15945101,  0.34390786, ...,  0.08609   ,
         0.76303923, -0.13470528],
       [ 1.0719687 , -0.6746578 ,  0.5029022 , ..., -0.03767636,
        -0.7001958 ,  0.1292309 ],
       ...,
       [-0.15600407,  0.10299437,  0.4183066 , ...,  0.662444  ,
        -0.3682016 ,  0.23117274],
       [ 0.00289336, -0.316857  ,  0.57558674, ...,  0.3563614 ,
        -0.00232075,  0.47726914],
       [-0.63912445, -0.2957042 ,  0.00930499, ...,  0.53227013,
        -0.20714217,  0.23913102]], dtype=float32)

In [18]:
X_train, X_val, Y_train, Y_val = train_test_split(x_embeddings, y, test_size=0.2, random_state=42)



# Build Model XGBOOST

In [19]:
model = XGBClassifier(n_estimators=100, max_depth=5, learning_rate=0.1)
model.fit(X_train, Y_train)

XGBClassifier(max_depth=5, objective='multi:softprob')

In [20]:
pred_val =model.predict(X_val)

In [21]:
print(confusion_matrix(Y_val, pred_val))
print(classification_report(Y_val, pred_val))

[[25  0  4 ...  0  0  0]
 [ 1 20  2 ...  0  0  0]
 [ 2  1 22 ...  0  0  0]
 ...
 [ 0  0  0 ... 26  0  0]
 [ 0  0  0 ...  0 29  0]
 [ 0  0  0 ...  0  0 29]]
              precision    recall  f1-score   support

           0       0.89      0.86      0.88        29
           1       0.95      0.87      0.91        23
           2       0.71      0.76      0.73        29
           3       0.83      0.83      0.83        12
           4       0.83      0.87      0.85        23
           5       0.95      0.75      0.84        24
           6       0.73      0.77      0.75        31
           7       0.96      0.87      0.92        31
           8       0.68      0.74      0.70        34
           9       0.89      0.89      0.89        35
          10       0.78      0.54      0.64        13
          11       0.74      0.93      0.82        30
          12       0.92      0.92      0.92        13
          13       0.94      0.97      0.95        30
          14       0.72      0.87

In [22]:
adjusted_rand = adjusted_rand_score(Y_val, pred_val)
adjusted_mutual_info = adjusted_mutual_info_score(Y_val, pred_val)

print("ARI: {}".format(adjusted_rand))
print("AMI: {}".format(adjusted_mutual_info))

ARI: 0.723218995040472
AMI: 0.803879810955724


In [23]:
x_test_embeddings = embedder.encode(X_test, convert_to_numpy=True)
pred_test =model.predict(x_test_embeddings)

In [24]:
my_submission = pd.DataFrame({'Id': df_test['id'], 'label': pred_test})
my_submission['label'] = my_submission['label'].map(cat_map)
print(my_submission.head())
my_submission.to_csv('Answers_SentTransformer_XGBoost.csv', index=False)

NameError: ignored

Build Model Naive Baise

In [ ]:
model = GaussianNB()
model.fit(X_train, Y_train)

In [ ]:
pred_val =model.predict(X_val)

In [ ]:
print(confusion_matrix(Y_val, pred_val))
print(classification_report(Y_val, pred_val))

In [ ]:
adjusted_rand = adjusted_rand_score(Y_val, pred_val)
adjusted_mutual_info = adjusted_mutual_info_score(Y_val, pred_val)

print("ARI: {}".format(adjusted_rand))
print("AMI: {}".format(adjusted_mutual_info))

In [ ]:
x_test_embeddings = embedder.encode(X_test, convert_to_numpy=True)
pred_test =model.predict(x_test_embeddings)

In [ ]:
my_submission = pd.DataFrame({'Id': df_test['id'], 'label': pred_test})
my_submission['label'] = my_submission['label'].map(cat_map)
print(my_submission.head())
my_submission.to_csv('Answers_SentTransformer_NB.csv', index=False)

Build Model Random Forest Classifier

In [ ]:
estimator = RandomForestClassifier(n_estimators=300, max_depth=10)
estimator.fit(X_train, Y_train)

In [ ]:
pred_val =model.predict(X_val)

In [ ]:
print(confusion_matrix(Y_val, pred_val))
print(classification_report(Y_val, pred_val))

In [ ]:
adjusted_rand = adjusted_rand_score(Y_val, pred_val)
adjusted_mutual_info = adjusted_mutual_info_score(Y_val, pred_val)

print("ARI: {}".format(adjusted_rand))
print("AMI: {}".format(adjusted_mutual_info))

In [ ]:
x_test_embeddings = embedder.encode(X_test, convert_to_numpy=True)
pred_test =model.predict(x_test_embeddings)

In [ ]:
my_submission = pd.DataFrame({'Id': df_test['id'], 'label': pred_test})
my_submission['label'] = my_submission['label'].map(cat_map)
print(my_submission.head())
my_submission.to_csv('Answers_SentTransformer_RF.csv', index=False)

Build Model CatBoost

In [ ]:
model = CatBoostClassifier(n_estimators=111, max_depth=7, learning_rate=0.01)
model.fit(X_train, Y_train)

In [ ]:
pred_val =model.predict(X_val)

In [ ]:
print(confusion_matrix(Y_val, pred_val))
print(classification_report(Y_val, pred_val))

In [57]:
adjusted_rand = adjusted_rand_score(Y_val, pred_val)
adjusted_mutual_info = adjusted_mutual_info_score(Y_val, pred_val)

print("ARI: {}".format(adjusted_rand))
print("AMI: {}".format(adjusted_mutual_info))

ValueError: ignored

In [58]:
x_test_embeddings = embedder.encode(X_test, convert_to_numpy=True)
pred_test =model.predict(x_test_embeddings)

In [59]:
my_submission = pd.DataFrame({'Id': df_test['id'], 'label': pred_test})
my_submission['label'] = my_submission['label'].map(cat_map)
print(my_submission.head())
my_submission.to_csv('Answers_SentTransformer_CatBoost.csv', index=False)

ValueError: ignored